### 1. Load CATPCHA images and get the transformed (X,y) data

In [1]:
import torch

from helper_functions.data_transformation import get_transformed_data_v2, get_transformed_data_for_captcha_evaluation

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"
cached_data_path = "../data/prepared_data_cnn_v2.pth"

try:
    # Load the prepared data from cache if it already exists
    X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, num_failed_segmentations = torch.load(cached_data_path)
    print("Loaded prepared data from cache.")

except FileNotFoundError:
    # Else prepare the data and save it
    X_train, y_train = get_transformed_data_v2(train_folder, is_train=True)
    X_test, y_test = get_transformed_data_v2(test_folder, is_train=False)
    X_test_captcha, y_test_captcha, num_failed_segmentations = get_transformed_data_for_captcha_evaluation(test_folder)

    torch.save((X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, num_failed_segmentations), cached_data_path)
    print("Saved prepared data to cache.")


Loaded prepared data from cache.


### 2. Instantiate basic CNN model and train it

In [ ]:
from models.cnn_v2 import CNN, CNNParams
from helper_functions.data_transformation import IMG_HEIGHT, IMG_WIDTH


# Instantiate model
cnn_params: CNNParams = {
    "num_classes": 36,
    "input_shape": (1, IMG_HEIGHT, IMG_WIDTH), # (channels, height, width)
    "learning_rate": 1.1*1e-3,
    "num_epochs": 10,
    "batch_size": 500,
}
cnn = CNN(cnn_params)

# Train model
cached_model_path = "../trained_models/cnn_v2.pth"
try:
    # Load the trained model from cache if it already exists
    cnn.load_trained_model(cached_model_path)
    print("Loaded trained model from cache.")
except FileNotFoundError:
    # Else train the model and save it
    cnn.train_model(X_train, y_train)
    cnn.save_trained_model(cached_model_path)
    print("Saved trained model to cache.")


Loaded trained model from cache.


### 3. Evaluate CNN model

In [7]:
import json
from helper_functions.evaluation import evaluate_character_performance, evaluate_captcha_performance

character_performance = evaluate_character_performance(cnn.model, X_test, y_test)
print(f"Character level performance (skipping failed segmentations): {json.dumps(character_performance, indent=2)}\n")

captcha_performance = evaluate_captcha_performance(cnn.model, X_test_captcha, y_test_captcha, num_failed_segmentations)
print(f"Captcha level performance (including failed segmentations): {json.dumps(captcha_performance, indent=2)}")

Character level performance (skipping failed segmentations): {
  "accuracy": 0.8502528199144301,
  "precision": 0.8535853610955613,
  "recall": 0.8468230654619362,
  "f1_score": 0.8487479860664281
}

Captcha level performance (including failed segmentations): {
  "captcha_accuracy": 0.35744456177402323,
  "character_accuracy": 0.7828665888759238,
  "segmentation_accuracy": 0.9091869060190074
}
